## Week 3 - Assignment - Add GeoData

### Neighbourhoods in Toronto

<b>Import the required libraries</b>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

<b>Get web page containing data</b>

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  # URL containing table
page = requests.get(url)  # pull page

<b>Parse web page for table</b>

In [3]:
soup = BeautifulSoup(page.content)

table = soup.find('table',{'class':'wikitable sortable'})  # find table in page

column_names = [header.text[:-1] for header in table.find_all('th')]  # find column names

row_data = table.find_all('tr')  # find rows
rows = list()
for row in row_data:
    cells = row.find_all('td')
    value = [cell.text[:-1] for cell in cells]
    rows.append(value)

<b>Convert table data into a Dataframe</b>

In [4]:
table_data = pd.DataFrame(rows[1:])
table_data.columns = column_names
table_data.tail()

,Postal Code,Borough,Neighbourhood
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


<b>Clean and process the Dataframe</b>

In [5]:
# Step A: Drop unassigned Boroughs
mask = table_data['Borough'] == 'Not assigned'
stepA = table_data[~mask]

# Step B: Expand Neighbourhood column
# stepB = pd.concat([stepA, stepA['Neighbourhood'].str.split(',', expand=True)], axis=1)
# stepB.drop('Neighbourhood', inplace=True, axis=1)

# Step C: Re-assign Postal code and Borough to Neighbourhoods
# stepC = stepB.melt(id_vars=['Postal Code', 'Borough'], value_name='Neighbourhood', value_vars=[0, 1, 2, 3, 4, 5, 6, 7])
# stepC.drop('variable', axis=1, inplace=True)

# Step D: Drop any rows with empty values
stepD = stepA.dropna()
stepD.reset_index(drop=True, inplace=True)
cleaned_df = stepD

In [6]:
cleaned_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
print(f'The table has {cleaned_df.shape[0]} rows and {cleaned_df.shape[1]} columns')

The table has 103 rows and 3 columns


<b>Get Coordinates from CSV</b>

In [8]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')  # read GeoData CSV
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
print(f'The table has {geodata.shape[0]} rows and {geodata.shape[1]} columns')

The table has 103 rows and 3 columns


<b>Merge Cleaned Dataframe with GeoData</b>

In [10]:
df = cleaned_df.merge(geodata, how='left', left_on='Postal Code', right_on='Postal Code')  # perform a left join
df.tail()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [11]:
print(f'The table has {df.shape[0]} rows and {df.shape[1]} columns')

The table has 103 rows and 5 columns
